In [1]:
'''
This file takes S&P500 data (both the individual stocks and index) from yahoo finance and 
calculates the market return, market standard deviation, 
stock return, stock standard deviation, as well as stock return for the past 12 months less the recent 1/2/3/6 months.
It also calculates the dollar value (dv) (volume * closing price) change of each stock over the rolling 1/2/3/6/12 months.
Finally, it also calculates the % of positive returns over the past 1/2/3/6/12 months.
It then combines all the dataframes into a single dataframe, with dates and tickers as the rows and features as the columns.

df contains raw data downloaded from yfinance
df1 contains close data for all S&P 500 stocks and the index
daily_return_df contains the daily return data for all S&P 500 stocks and the index
M1_return_df contains the monthly return data for all S&P 500 stocks and the index
M2_return_df contains the 2 month return data for all S&P 500 stocks and the index
M3_return_df contains the 3 month return data for all S&P 500 stocks and the index
M6_return_df contains the 6 month return data for all S&P 500 stocks and the index
M12_return_df contains the 12 month return data for all S&P 500 stocks and the index

M1_forward_return_df contains the 1 month forward return data for all S&P 500 stocks and the index

M1_sd_df contains the 1 month standard deviation data for all S&P 500 stocks and the index
M2_sd_df contains the 2 month standard deviation data for all S&P 500 stocks and the index
M3_sd_df contains the 3 month standard deviation data for all S&P 500 stocks and the index
M6_sd_df contains the 6 month standard deviation data for all S&P 500 stocks and the index
M12_sd_df contains the 12 month standard deviation data for all S&P 500 stocks and the index

M12_1_return_df contains the 12 month return data minus 1 month return data for all S&P 500 stocks and the index
M12_2_return_df contains the 12 month return data minus 2 month return data for all S&P 500 stocks and the index
M12_3_return_df contains the 12 month return data minus 3 month return data for all S&P 500 stocks and the index
M12_6_return_df contains the 12 month return data minus 6 month return data for all S&P 500 stocks and the index

dv_df contains all the dollar value for all S&P 500 stocks and the index
dv_change_df contains the daily change in dv
M1_dv_change_df contains the monthly change in dv
M2_dv_change_df contains the 2 month change in dv
M3_dv_change_df contains the 3 month change in dv
M6_dv_change_df contains the 6 month change in dv
M12_dv_change_df contains the 12 month change in dv

M1_positive_returns_df contains the percentage of positive returns over past 1 month
M2_positive_returns_df contains the percentage of positive returns over past 2 months
M3_positive_returns_df contains the percentage of positive returns over past 3 months
M6_positive_returns_df contains the percentage of positive returns over past 6 months
M12_positive_returns_df contains the percentage of positive returns over past 12 months
'''

'\nThis file takes S&P500 data (both the individual stocks and index) from yahoo finance and \ncalculates the market return, market standard deviation, \nstock return, stock standard deviation, as well as stock return for the past 12 months less the recent 1/2/3/6 months.\nIt also calculates the dollar value (dv) (volume * closing price) change of each stock over the rolling 1/2/3/6/12 months.\nFinally, it also calculates the % of positive returns over the past 1/2/3/6/12 months.\nIt then combines all the dataframes into a single dataframe, with dates and tickers as the rows and features as the columns.\n\ndf contains raw data downloaded from yfinance\ndf1 contains close data for all S&P 500 stocks and the index\ndaily_return_df contains the daily return data for all S&P 500 stocks and the index\nM1_return_df contains the monthly return data for all S&P 500 stocks and the index\nM2_return_df contains the 2 month return data for all S&P 500 stocks and the index\nM3_return_df contains th

In [100]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycaret

# Yahoo finance
import yfinance as yf

# Datetime
import datetime as dt
from datetime import datetime

In [101]:
start=pd.to_datetime('1990-01-01', format='%Y-%m-%d')
today_date = datetime.today().strftime('%Y-%m-%d')
period=256           #lookback period for calculation of beta, realized returns
period_1m=21
period_2m=42
period_3m=63
period_6m=128
period_12m=256


In [102]:
#Download data for multiple indicators available on Yahoo Finance

# Read and print the stock tickers that make up S&P500
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']
tickers=tickers.to_list()

print(tickers)
#print(tickers.head())

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'C

In [103]:
#Two of the tickers have issue, also adding in S&P 500 index
for i in range(len(tickers)):
 
    # replace BF.B with BF-B
    if tickers[i] == 'BF.B':
        tickers[i] = 'BF-B'
 
    # replace BRK.B with BRK-B
    if tickers[i] == 'BRK.B':
        tickers[i] = 'BRK-B'

#Add S&P index
tickers.append("^GSPC")
# print list
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'C

In [104]:
tickers = yf.Tickers(tickers)

In [105]:
df = tickers.download(start=start,end=today_date,interval='1D',)   #['Adj Close']
#df contains raw data downloaded from yfinance
df

[*********************100%%**********************]  504 of 504 completed


Price            Close                                                         \
Ticker               A    AAL        AAPL        ABBV        ABNB         ABT   
Date                                                                            
1990-01-02         NaN    NaN    0.262721         NaN         NaN    1.809341   
1990-01-03         NaN    NaN    0.264485         NaN         NaN    1.815873   
1990-01-04         NaN    NaN    0.265367         NaN         NaN    1.812606   
1990-01-05         NaN    NaN    0.266248         NaN         NaN    1.793012   
1990-01-08         NaN    NaN    0.268012         NaN         NaN    1.802202   
...                ...    ...         ...         ...         ...         ...   
2024-06-06  132.820007  11.43  194.479996  168.570007  147.000000  104.269997   
2024-06-07  132.979996  11.50  196.889999  169.419998  146.660004  107.580002   
2024-06-10  133.029999  11.49  193.119995  170.460007  148.070007  107.489998   
2024-06-11  132.110001  11.28  207.149994  167.699997  147.750000  105.809998   
2024-06-12  133.160004  11.50  213.070007  166.089996  149.320007  104.379997   

Price                                                       ...   Volume  \
Ticker            ACGL         ACN        ADBE         ADI  ...       WY   
Date                                                        ...            
1990-01-02         NaN         NaN    1.188340    1.007657  ...   442400   
1990-01-03         NaN         NaN    1.247023    0.955310  ...   289700   
1990-01-04         NaN         NaN    1.305707    0.942225  ...   324200   
1990-01-05         NaN         NaN    1.335049    0.942225  ...   214000   
1990-01-08         NaN         NaN    1.352692    0.942225  ...   275400   
...                ...         ...         ...         ...  ...      ...   
2024-06-06  100.080002  291.739990  458.130005  237.410004  ...  4555100   
2024-06-07  100.570000  288.399994  465.429993  234.770004  ...  3397900   
2024-06-10  100.580002  290.429993  459.940002  235.369995  ...  4424000   
2024-06-11   99.580002  294.220001  462.690002  236.300003  ...  3152800   
2024-06-12   99.160004  285.730011  459.869995  238.440002  ...  3590200   

Price                                                                      \
Ticker           WYNN      XEL       XOM        XYL        YUM        ZBH   
Date                                                                        
1990-01-02        NaN   247200   5326000        NaN        NaN        NaN   
1990-01-03        NaN   126800   4980400        NaN        NaN        NaN   
1990-01-04        NaN   204200   6013200        NaN        NaN        NaN   
1990-01-05        NaN   144800   3854800        NaN        NaN        NaN   
1990-01-08        NaN   189000   4302000        NaN        NaN        NaN   
...               ...      ...       ...        ...        ...        ...   
2024-06-06  1194600.0  2395000  13154100  1190300.0  1528900.0   902300.0   
2024-06-07  1416200.0  3824500  13289200   873900.0  1467100.0  1070300.0   
2024-06-10  1395500.0  4067900  15245200  1304100.0  1717000.0  1214500.0   
2024-06-11  2077500.0  3273600  12462400   725400.0  2149700.0  3036000.0   
2024-06-12  2039100.0  3006100  13603800   656400.0  1276000.0  2870300.0   

Price                                        
Ticker          ZBRA        ZTS       ^GSPC  
Date                                         
1990-01-02       NaN        NaN   162070000  
1990-01-03       NaN        NaN   192330000  
1990-01-04       NaN        NaN   177000000  
1990-01-05       NaN        NaN   158530000  
1990-01-08       NaN        NaN   140110000  
...              ...        ...         ...  
2024-06-06  162800.0  2722200.0  3609990000  
2024-06-07  187300.0  1819400.0  3692760000  
2024-06-10  227800.0  2506900.0  3622280000  
2024-06-11  172900.0  1844100.0  3568030000  
2024-06-12  189200.0  2067800.0  3962840000  

[8678 rows x 3528 columns]

In [106]:
list(df.columns.levels[0])

['Close', 'Dividends', 'High', 'Low', 'Open', 'Stock Splits', 'Volume']

In [107]:
df1=df['Close']

In [108]:
#df1 contains close data for all S&P 500 stocks and the index
df1

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,0.262721,NaN,NaN,1.809341,NaN,NaN,1.188340,1.007657,...,3.252944,NaN,4.096442,3.929668,NaN,NaN,NaN,NaN,NaN,359.690002
1990-01-03,NaN,NaN,0.264485,NaN,NaN,1.815873,NaN,NaN,1.247023,0.955310,...,3.181295,NaN,4.083798,3.890373,NaN,NaN,NaN,NaN,NaN,358.760010
1990-01-04,NaN,NaN,0.265367,NaN,NaN,1.812606,NaN,NaN,1.305707,0.942225,...,3.152634,NaN,3.982653,3.851075,NaN,NaN,NaN,NaN,NaN,355.670013
1990-01-05,NaN,NaN,0.266248,NaN,NaN,1.793012,NaN,NaN,1.335049,0.942225,...,3.138305,NaN,3.944724,3.831427,NaN,NaN,NaN,NaN,NaN,352.200012
1990-01-08,NaN,NaN,0.268012,NaN,NaN,1.802202,NaN,NaN,1.352692,0.942225,...,3.138305,NaN,3.906792,3.890373,NaN,NaN,NaN,NaN,NaN,353.790009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,132.820007,11.43,194.479996,168.570007,147.000000,104.269997,100.080002,291.739990,458.130005,237.410004,...,29.760000,92.480003,54.820000,113.970001,136.869995,140.830002,113.370003,307.489990,176.779999,5352.959961
2024-06-07,132.979996,11.50,196.889999,169.419998,146.660004,107.580002,100.570000,288.399994,465.429993,234.770004,...,29.360001,93.139999,54.740002,112.750000,137.320007,139.970001,113.339996,300.829987,176.919998,5346.990234
2024-06-10,133.029999,11.49,193.119995,170.460007,148.070007,107.489998,100.580002,290.429993,459.940002,235.369995,...,29.370001,92.870003,55.240002,113.080002,139.250000,137.809998,112.879997,302.600006,178.830002,5360.790039


In [109]:
#Calculate the daily returns
daily_return_df = np.log(df1/df1.shift(1))

In [110]:
#daily_return_df contains the daily return data for all S&P 500 stocks and the index
daily_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,0.006688,NaN,NaN,0.003604,NaN,NaN,0.048202,-0.053347,...,-0.022272,NaN,-0.003091,-0.010050,NaN,NaN,NaN,NaN,NaN,-0.002589
1990-01-04,NaN,NaN,0.003331,NaN,NaN,-0.001800,NaN,NaN,0.045985,-0.013792,...,-0.009050,NaN,-0.025079,-0.010153,NaN,NaN,NaN,NaN,NaN,-0.008650
1990-01-05,NaN,NaN,0.003316,NaN,NaN,-0.010869,NaN,NaN,0.022223,0.000000,...,-0.004555,NaN,-0.009569,-0.005115,NaN,NaN,NaN,NaN,NaN,-0.009804
1990-01-08,NaN,NaN,0.006601,NaN,NaN,0.005113,NaN,NaN,0.013129,0.000000,...,0.000000,NaN,-0.009662,0.015268,NaN,NaN,NaN,NaN,NaN,0.004504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,-0.005556,-0.016486,-0.007122,0.018743,0.008334,0.010024,0.004005,0.002471,0.005099,0.007314,...,0.010471,-0.009471,-0.006183,0.007486,-0.020251,0.002133,0.006460,-0.000033,0.005445,-0.000200
2024-06-07,0.001204,0.006106,0.012316,0.005030,-0.002316,0.031251,0.004884,-0.011515,0.015809,-0.011182,...,-0.013532,0.007111,-0.001460,-0.010762,0.003282,-0.006125,-0.000265,-0.021897,0.000792,-0.001116
2024-06-10,0.000376,-0.000870,-0.019333,0.006120,0.009568,-0.000837,0.000099,0.007014,-0.011866,0.002552,...,0.000341,-0.002903,0.009093,0.002923,0.013957,-0.015552,-0.004067,0.005867,0.010738,0.002578


In [111]:
#Calculate the monthly returns
M1_return_df = np.log(df1/df1.shift(period_1m))

In [112]:
#M1_return_df contains the monthly return data for all S&P 500 stocks and the index
M1_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,-0.061678,-0.220515,0.065483,0.036365,-0.083553,-0.018058,0.018862,-0.062836,-0.071874,0.157748,...,-0.029219,-0.047509,-0.003642,-0.011030,-0.020212,0.040463,-0.068681,-0.033200,0.048267,0.031359
2024-06-07,-0.071246,-0.226269,0.075936,0.054398,-0.073845,0.024846,0.016441,-0.078623,-0.047559,0.140249,...,-0.038854,-0.025059,-0.003465,-0.021620,-0.025250,0.026641,-0.064158,-0.061619,0.070237,0.030249
2024-06-10,-0.087124,-0.231291,0.046638,0.060830,0.006912,0.026585,0.009490,-0.054214,-0.048196,0.143143,...,-0.051446,-0.042889,0.000000,-0.038221,-0.021230,0.003887,-0.073095,-0.044911,0.061698,0.027749


In [113]:
#Calculate the M2 returns
M2_return_df = np.log(df1/df1.shift(period_2m))

In [114]:
#M2_return_df contains the 2 month return data for all S&P 500 stocks and the index
M2_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,-0.084008,-0.195647,0.145046,0.002026,-0.086427,-0.053266,0.056841,-0.124805,-0.055510,0.192129,...,-0.138720,-0.133775,0.013591,-0.048040,0.071569,0.045553,-0.111086,0.035148,0.067635,0.028532
2024-06-07,-0.102951,-0.193132,0.150145,0.005055,-0.103545,-0.039238,0.097580,-0.144274,-0.056634,0.143868,...,-0.169819,-0.131370,0.006782,-0.064014,0.065658,0.026203,-0.132516,-0.017595,0.059713,0.025971
2024-06-10,-0.080349,-0.154524,0.142014,0.017720,-0.079237,-0.028716,0.087102,-0.110615,-0.057620,0.185332,...,-0.127800,-0.112917,0.027344,-0.069474,0.085370,0.003887,-0.123515,0.008929,0.094306,0.038051


In [115]:
#Calculate the M3 returns
M3_return_df = np.log(df1/df1.shift(period_3m))

In [116]:
#M3_return_df contains the 3 month return data for all S&P 500 stocks and the index
M3_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,-0.115407,-0.265120,0.141786,-0.059472,-0.106625,-0.143198,0.132162,-0.278465,-0.193687,0.184280,...,-0.137267,-0.086639,0.101930,0.067744,0.077694,0.008103,-0.114482,0.063436,-0.023557,0.037225
2024-06-07,-0.104512,-0.244139,0.143917,-0.044871,-0.117283,-0.112278,0.137959,-0.267131,-0.170025,0.184768,...,-0.164381,-0.072780,0.081075,0.047619,0.080110,0.007836,-0.109867,0.062548,-0.026341,0.042659
2024-06-10,-0.100205,-0.242963,0.112822,-0.043103,-0.096004,-0.106729,0.107705,-0.246941,-0.197590,0.175801,...,-0.170620,-0.103181,0.053793,0.044654,0.101174,-0.011436,-0.124139,0.076801,-0.022877,0.046359


In [117]:
#Calculate the M6 returns
M6_return_df = np.log(df1/df1.shift(period_6m))

In [118]:
#M6_return_df contains the 6 month return data for all S&P 500 stocks and the index
M6_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.034125,-0.130245,0.019431,0.180448,0.085010,0.003933,0.191597,-0.139851,-0.290347,0.268591,...,-0.047609,0.087523,-0.094757,0.118644,0.259233,0.110665,-0.035902,0.242733,-0.008093,0.152761
2024-06-07,0.034630,-0.149169,0.041256,0.180331,0.092519,0.032233,0.204257,-0.146532,-0.261539,0.256371,...,-0.056087,0.109542,-0.096868,0.113334,0.259976,0.117822,-0.026970,0.228535,-0.023855,0.157069
2024-06-10,0.042795,-0.133423,0.001078,0.183542,0.102012,0.039414,0.206065,-0.137733,-0.269527,0.273379,...,-0.046521,0.126869,-0.080592,0.135875,0.280340,0.112429,-0.021841,0.255553,0.000538,0.160215


In [119]:
#Calculate the M12 returns
M12_return_df = np.log(df1/df1.shift(period_12m))

In [120]:
#M12_return_df contains the 12 month return data for all S&P 500 stocks and the index
M12_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.145489,-0.257033,0.098065,0.240177,0.292045,0.042082,0.361770,-0.032190,0.092174,0.312640,...,0.072886,-0.054597,-0.138441,0.143806,0.323888,0.109434,-0.108502,0.157925,0.090796,0.247379
2024-06-07,0.141605,-0.245500,0.094485,0.278518,0.268191,0.065326,0.367084,-0.041348,0.086763,0.286266,...,0.056899,-0.024737,-0.112889,0.121562,0.308092,0.076021,-0.108924,0.122297,0.073411,0.236457
2024-06-10,0.125262,-0.257869,0.070388,0.259259,0.226492,0.051157,0.349683,-0.054413,0.052606,0.287427,...,0.047129,-0.066968,-0.097324,0.101530,0.293764,0.043376,-0.132045,0.084755,0.049340,0.224605


In [121]:
#Calculate the 1 month forward returns
M1_forward_return_df = np.log(df1/df1.shift(-period_1m))

In [122]:
#M1_forward_return_df contains the 1 month forward return data for all S&P 500 stocks and the index
M1_forward_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,0.091292,NaN,NaN,0.058223,NaN,NaN,-0.151046,0.232932,...,0.139630,NaN,0.083650,0.061875,NaN,NaN,NaN,NaN,NaN,0.088942
1990-01-03,NaN,NaN,0.109071,NaN,NaN,0.057988,NaN,NaN,-0.113426,0.196114,...,0.112242,NaN,0.083920,0.051825,NaN,NaN,NaN,NaN,NaN,0.087234
1990-01-04,NaN,NaN,0.093983,NaN,NaN,0.048554,NaN,NaN,-0.088275,0.182322,...,0.082990,NaN,0.055480,0.023228,NaN,NaN,NaN,NaN,NaN,0.072127
1990-01-05,NaN,NaN,0.075638,NaN,NaN,0.041494,NaN,NaN,-0.066052,0.182322,...,0.083447,NaN,0.056029,0.015504,NaN,NaN,NaN,NaN,NaN,0.059516
1990-01-08,NaN,NaN,0.089407,NaN,NaN,0.058123,NaN,NaN,-0.042560,0.216224,...,0.098637,NaN,0.049762,0.033942,NaN,NaN,NaN,NaN,NaN,0.070642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
#Calculate the M1 standard deviation
M1_sd_df=daily_return_df.rolling(period_1m).std()

In [124]:
#M1_sd_df contains the 1 month standard deviation data for all S&P 500 stocks and the index
M1_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.026546,0.034916,0.008825,0.013031,0.019361,0.008749,0.010930,0.011893,0.018172,0.025310,...,0.010320,0.011733,0.012174,0.013344,0.013264,0.011016,0.009438,0.012002,0.011430,0.005297
2024-06-07,0.026380,0.034753,0.009043,0.012598,0.019276,0.010858,0.010872,0.011910,0.018601,0.025638,...,0.010646,0.011502,0.012173,0.013528,0.013132,0.011067,0.009453,0.012628,0.010102,0.005318
2024-06-10,0.026014,0.034663,0.010200,0.012602,0.011645,0.010834,0.010777,0.011700,0.018617,0.025606,...,0.010113,0.010895,0.012278,0.012729,0.013331,0.011560,0.009278,0.012632,0.009587,0.005261


In [125]:
#Calculate the M2 standard deviation
M2_sd_df=daily_return_df.rolling(period_2m).std()

In [126]:
#M2_sd_df contains the 2 month standard deviation data for all S&P 500 stocks and the index
M2_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.022088,0.031723,0.015310,0.013513,0.019114,0.009684,0.013498,0.011669,0.015818,0.023459,...,0.013123,0.016444,0.010903,0.011697,0.012430,0.011362,0.011366,0.017972,0.020905,0.007288
2024-06-07,0.021817,0.031741,0.015361,0.013532,0.018927,0.010541,0.012156,0.011611,0.015787,0.023006,...,0.012789,0.016476,0.010877,0.011745,0.012376,0.011249,0.010735,0.017664,0.020875,0.007292
2024-06-10,0.021595,0.031262,0.015601,0.013521,0.018915,0.010412,0.012089,0.011121,0.015802,0.022013,...,0.011334,0.016222,0.010803,0.011662,0.012465,0.011477,0.010621,0.017394,0.020533,0.007119


In [127]:
#Calculate the M3 standard deviation
M3_sd_df=daily_return_df.rolling(period_3m).std()

In [128]:
#M3_sd_df contains the 3 month standard deviation data for all S&P 500 stocks and the index
M3_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.019074,0.029071,0.014711,0.014037,0.017538,0.010371,0.013265,0.016865,0.024242,0.020740,...,0.012130,0.015839,0.011956,0.010921,0.011030,0.010800,0.011550,0.017153,0.018489,0.006940
2024-06-07,0.019051,0.029067,0.014732,0.014013,0.017491,0.011195,0.013264,0.016725,0.024350,0.020735,...,0.012042,0.015859,0.011743,0.010969,0.011033,0.010803,0.011545,0.017172,0.018483,0.006883
2024-06-10,0.019050,0.029068,0.014929,0.014025,0.017503,0.011180,0.012767,0.016744,0.024261,0.020705,...,0.011991,0.015430,0.010901,0.010953,0.011094,0.010971,0.011446,0.017140,0.018511,0.006884


In [129]:
#Calculate the M6 standard deviation
M6_sd_df=daily_return_df.rolling(period_6m).std()

In [130]:
#M6_sd_df contains the 6 month standard deviation data for all S&P 500 stocks and the index
M6_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.017557,0.028659,0.013364,0.011810,0.019202,0.010187,0.013184,0.014605,0.022249,0.018476,...,0.013849,0.017028,0.016090,0.011552,0.011238,0.010094,0.011346,0.023330,0.017692,0.006937
2024-06-07,0.017557,0.028572,0.013379,0.011809,0.019181,0.010552,0.013162,0.014630,0.022285,0.018513,...,0.013891,0.016981,0.016090,0.011585,0.011239,0.010034,0.011318,0.023409,0.017630,0.006915
2024-06-10,0.017542,0.028539,0.013365,0.011816,0.019197,0.010526,0.013159,0.014648,0.022301,0.018454,...,0.013870,0.016880,0.016103,0.011441,0.011262,0.010092,0.011295,0.023323,0.017615,0.006914


In [131]:
#Calculate the M12 standard deviation
M12_sd_df=daily_return_df.rolling(period_12m).std()

In [132]:
#M12_sd_df contains the 12 month standard deviation data for all S&P 500 stocks and the index
M12_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.017148,0.024473,0.012686,0.011780,0.021280,0.011169,0.014052,0.013440,0.020343,0.016908,...,0.013519,0.017763,0.014219,0.013047,0.012365,0.010063,0.013285,0.025350,0.016875,0.007169
2024-06-07,0.017146,0.024476,0.012671,0.011585,0.021242,0.011326,0.014053,0.013458,0.020324,0.016903,...,0.013546,0.017713,0.014122,0.013048,0.012315,0.009929,0.013285,0.025375,0.016838,0.007149
2024-06-10,0.017116,0.024463,0.012728,0.011488,0.021014,0.011297,0.014017,0.013405,0.020292,0.016903,...,0.013531,0.017540,0.014129,0.012973,0.012224,0.009922,0.013231,0.025234,0.016712,0.007099


In [133]:
#Calculate the M12-M1 return
M12_1_return_df=M12_return_df-M1_return_df

In [134]:
#M12_1_return_df contains the 12 month return data minus 1 month return data for all S&P 500 stocks and the index
M12_1_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.207167,-0.036518,0.032582,0.203811,0.375598,0.060140,0.342908,0.030647,0.164048,0.154892,...,0.102106,-0.007088,-0.134799,0.154836,0.344100,0.068971,-0.039821,0.191125,0.042528,0.216020
2024-06-07,0.212851,-0.019231,0.018549,0.224120,0.342035,0.040480,0.350643,0.037275,0.134322,0.146017,...,0.095752,0.000322,-0.109424,0.143182,0.333342,0.049380,-0.044766,0.183917,0.003174,0.206208
2024-06-10,0.212387,-0.026577,0.023749,0.198429,0.219580,0.024572,0.340193,-0.000199,0.100801,0.144284,...,0.098575,-0.024079,-0.097324,0.139752,0.314993,0.039489,-0.058950,0.129666,-0.012358,0.196856


In [135]:
#Calculate the M12-M2 return
M12_2_return_df=M12_return_df-M2_return_df

In [136]:
#M12_2_return_df contains the 12 month return data minus 2 month return data for all S&P 500 stocks and the index
M12_2_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.229496,-0.061386,-0.046981,0.238151,0.378473,0.095348,0.304929,0.092615,0.147684,0.120511,...,0.211607,0.079177,-0.152031,0.191846,0.252318,0.063882,0.002584,0.122777,0.023161,0.218848
2024-06-07,0.244556,-0.052368,-0.055660,0.273464,0.371736,0.104564,0.269504,0.102927,0.143398,0.142398,...,0.226718,0.106633,-0.119671,0.185577,0.242434,0.049818,0.023593,0.139893,0.013698,0.210486
2024-06-10,0.205612,-0.103345,-0.071626,0.241539,0.305729,0.079873,0.262581,0.056202,0.110225,0.102096,...,0.174929,0.045949,-0.124668,0.171004,0.208393,0.039489,-0.008530,0.075825,-0.044966,0.186554


In [137]:
#Calculate the M12-M3 return
M12_3_return_df=M12_return_df-M3_return_df

In [138]:
#M12_3_return_df contains the 12 month return data minus 3 month return data for all S&P 500 stocks and the index
M12_3_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.260895,0.008086,-0.043721,0.299649,0.398670,0.185281,0.229608,0.246275,0.285861,0.128360,...,0.210153,0.032041,-0.240371,0.076062,0.246194,0.101332,0.005980,0.094489,0.114353,0.210154
2024-06-07,0.246116,-0.001361,-0.049432,0.323389,0.385473,0.177604,0.229125,0.225783,0.256788,0.101499,...,0.221279,0.048043,-0.193964,0.073943,0.227982,0.068185,0.000943,0.059749,0.099751,0.193798
2024-06-10,0.225468,-0.014905,-0.042434,0.302361,0.322496,0.157886,0.241978,0.192528,0.250196,0.111626,...,0.217749,0.036213,-0.151117,0.056877,0.192589,0.054811,-0.007906,0.007954,0.072217,0.178245


In [139]:
#Calculate the M12-M6 return
M12_6_return_df=M12_return_df-M6_return_df

In [140]:
#M12_6_return_df contains the 12 month return data minus 6 month return data for all S&P 500 stocks and the index
M12_6_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.111364,-0.126788,0.078634,0.059729,0.207036,0.038149,0.170173,0.107661,0.382521,0.044049,...,0.120495,-0.142120,-0.043684,0.025163,0.064655,-0.001231,-0.072600,-0.084808,0.098889,0.094618
2024-06-07,0.106975,-0.096331,0.053229,0.098187,0.175672,0.033093,0.162827,0.105184,0.348303,0.029895,...,0.112986,-0.134279,-0.016021,0.008228,0.048116,-0.041801,-0.081954,-0.106238,0.097266,0.079388
2024-06-10,0.082467,-0.124446,0.069309,0.075717,0.124480,0.011743,0.143618,0.083321,0.322132,0.014049,...,0.093651,-0.193837,-0.016732,-0.034345,0.013423,-0.069053,-0.110204,-0.170799,0.048802,0.064389


In [141]:
#Calculate dollar value by multiplying volume by closing position
dv_df=df['Close']*df['Volume']

In [142]:
#dv_df contains all the dollar value for all S&P 500 stocks and the index
dv_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,4.813015e+07,NaN,NaN,1.828603e+07,NaN,NaN,8.516119e+06,9.026588e+05,...,1.439103e+06,NaN,1.012640e+06,2.092941e+07,NaN,NaN,NaN,NaN,NaN,5.829496e+10
1990-01-03,NaN,NaN,5.501151e+07,NaN,NaN,1.110313e+07,NaN,NaN,1.685776e+07,6.247728e+05,...,9.216211e+05,NaN,5.178256e+05,1.937561e+07,NaN,NaN,NaN,NaN,NaN,6.900031e+10
1990-01-04,NaN,NaN,5.878241e+07,NaN,NaN,1.173892e+07,NaN,NaN,1.548464e+07,7.784661e+05,...,1.022084e+06,NaN,8.132577e+05,2.315728e+07,NaN,NaN,NaN,NaN,NaN,6.295359e+10
1990-01-05,NaN,NaN,3.283163e+07,NaN,NaN,6.064833e+06,NaN,NaN,8.822002e+06,2.747527e+05,...,6.715974e+05,NaN,5.711961e+05,1.476938e+07,NaN,NaN,NaN,NaN,NaN,5.583427e+10
1990-01-08,NaN,NaN,2.722270e+07,NaN,NaN,1.195096e+07,NaN,NaN,5.880423e+06,4.081718e+05,...,8.642893e+05,NaN,7.383837e+05,1.673638e+07,NaN,NaN,NaN,NaN,NaN,4.956952e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,3.014483e+08,4.096581e+08,8.009036e+09,8.743221e+08,5.090757e+08,1.025548e+09,1.133706e+08,6.894399e+08,1.482051e+09,6.545394e+08,...,1.355598e+08,1.104766e+08,1.312939e+08,1.499173e+09,1.629164e+08,2.153150e+08,1.022938e+08,5.005937e+07,4.812305e+08,1.932413e+13
2024-06-07,3.145376e+08,3.382311e+08,1.045563e+10,6.406956e+08,3.806707e+08,9.323528e+08,8.080799e+07,7.575979e+08,1.249307e+09,5.184191e+08,...,9.976235e+07,1.319049e+08,2.093531e+08,1.498357e+09,1.200040e+08,2.053500e+08,1.213078e+08,5.634546e+07,3.218882e+08,1.974515e+13
2024-06-10,2.226390e+08,2.691073e+08,1.878326e+10,7.240289e+08,4.561000e+08,9.197382e+08,9.612431e+07,7.289502e+08,1.477511e+09,6.509393e+08,...,1.299329e+08,1.296001e+08,2.247108e+08,1.723927e+09,1.815959e+08,2.366198e+08,1.370928e+08,6.893228e+07,4.483089e+08,1.941828e+13


In [143]:
#Calculate the daily dv change
dv_change_df = np.log(dv_df/dv_df.shift(1))
dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,0.133634,NaN,NaN,-0.498910,NaN,NaN,0.682850,-0.367956,...,-0.445641,NaN,-0.670678,-0.077140,NaN,NaN,NaN,NaN,NaN,0.168595
1990-01-04,NaN,NaN,0.066300,NaN,NaN,0.055682,NaN,NaN,-0.084963,0.219937,...,0.103465,NaN,0.451410,0.178294,NaN,NaN,NaN,NaN,NaN,-0.091713
1990-01-05,NaN,NaN,-0.582450,NaN,NaN,-0.660402,NaN,NaN,-0.562600,-1.041454,...,-0.419940,NaN,-0.353316,-0.449753,NaN,NaN,NaN,NaN,NaN,-0.120010
1990-01-08,NaN,NaN,-0.187341,NaN,NaN,0.678304,NaN,NaN,-0.405620,0.395817,...,0.252249,NaN,0.256731,0.125029,NaN,NaN,NaN,NaN,NaN,-0.119012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,-0.181978,0.026258,-0.281009,0.035502,0.112184,1.101403,-0.263412,-0.236858,-0.013278,-0.369004,...,0.621529,-0.120590,-0.039000,-0.114689,0.180812,-0.238067,-0.010899,-0.420939,0.380519,0.004946
2024-06-07,0.042505,-0.191593,0.266570,-0.310894,-0.290662,-0.095271,-0.338586,0.094273,-0.170837,-0.233148,...,-0.306622,0.177277,0.466584,-0.000544,-0.305712,-0.047386,0.170482,0.118292,-0.402142,0.021553
2024-06-10,-0.345552,-0.228619,0.585826,0.122277,0.180777,-0.013622,0.173566,-0.038547,0.167770,0.227632,...,0.264227,-0.017627,0.070792,0.140236,0.414259,0.141739,0.122327,0.201623,0.331278,-0.016693


In [144]:
#Calculate the monthly dv change
M1_dv_change_df = np.log(dv_df.rolling(period_1m).sum()/dv_df.shift(period_1m).rolling(period_1m).sum())
M1_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.491865,0.007537,-0.203826,-0.080619,0.292375,0.108217,0.206500,-0.030416,0.094450,0.216094,...,-0.015753,-0.041687,-0.165927,-0.227951,0.214981,-0.236995,-0.132239,-0.310163,-0.572147,0.045524
2024-06-07,0.490111,0.014426,-0.196990,-0.095725,0.147250,0.114026,0.220647,-0.023883,0.108016,0.249187,...,-0.019515,-0.222568,-0.165967,-0.255245,0.206363,-0.257991,-0.141181,-0.335750,-0.658188,0.039703
2024-06-10,0.488633,0.039743,-0.153196,-0.088532,-0.124210,0.121490,0.233241,-0.052047,0.123676,0.273865,...,0.014139,-0.259734,-0.183689,-0.263115,0.215845,-0.268270,-0.115899,-0.338844,-0.642390,0.040666


In [145]:
#Calculate the 3M dv change
M2_dv_change_df = np.log(dv_df.rolling(period_2m).sum()/dv_df.shift(period_2m).rolling(period_2m).sum())
M2_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,-0.069510,0.104325,-0.011258,-0.175854,-0.275836,-0.118723,0.063277,0.068642,-0.502870,0.089134,...,-0.127584,-0.197447,-0.470841,0.091316,0.293733,-0.059464,-0.155827,-0.069125,-0.059293,-0.055111
2024-06-07,-0.039903,0.106077,0.002357,-0.183295,-0.268168,-0.098111,0.029803,0.070683,-0.501199,0.065865,...,-0.115529,-0.158292,-0.459828,0.085243,0.354262,-0.036085,-0.146846,-0.078459,-0.054835,-0.045070
2024-06-10,-0.029693,0.073031,0.023744,-0.167688,-0.262003,-0.066801,0.032738,0.069171,-0.497607,0.044611,...,-0.111889,-0.088502,-0.456303,0.079608,0.378679,-0.036688,-0.084065,-0.084326,-0.059783,-0.043472


In [146]:
#Calculate the 3M dv change
M3_dv_change_df = np.log(dv_df.rolling(period_3m).sum()/dv_df.shift(period_3m).rolling(period_3m).sum())
M3_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.058986,-0.115836,-0.005009,0.085535,-0.194257,0.128864,0.213412,0.315775,-0.090056,0.078338,...,-0.143361,-0.175096,-0.225544,0.037922,0.119743,0.162294,-0.297013,-0.057739,0.340546,0.030186
2024-06-07,0.064343,-0.109982,-0.016208,0.071101,-0.202013,0.142917,0.215419,0.315414,-0.095568,0.067490,...,-0.151929,-0.166602,-0.252391,0.046253,0.122880,0.163727,-0.303926,-0.069680,0.282641,0.026957
2024-06-10,0.066373,-0.101806,-0.005527,0.070359,-0.193362,0.155854,0.205719,0.309435,-0.096826,0.069964,...,-0.142153,-0.178229,-0.286584,0.052951,0.124732,0.163308,-0.304374,-0.070839,0.262307,0.024599


In [147]:
#Calculate the 6M dv change
M6_dv_change_df = np.log(dv_df.rolling(period_6m).sum()/dv_df.shift(period_6m).rolling(period_6m).sum())
M6_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,-0.066782,0.182708,0.045038,0.251786,-0.157619,0.136140,0.028724,0.313242,0.126872,0.110714,...,0.010565,-0.166093,0.113943,0.099850,0.031692,0.250185,-0.069314,-0.097866,0.417561,0.166654
2024-06-07,-0.054371,0.170075,0.049828,0.260650,-0.160882,0.141402,0.026529,0.312183,0.127390,0.111925,...,0.007811,-0.153115,0.117697,0.093850,0.036725,0.247852,-0.071045,-0.098118,0.419832,0.165907
2024-06-10,-0.050737,0.164582,0.052819,0.265508,-0.155940,0.149961,0.028579,0.316618,0.128715,0.116510,...,0.015876,-0.148366,0.127361,0.088857,0.046004,0.251989,-0.067541,-0.093937,0.429295,0.167076


In [148]:
#Calculate the 12M dv change
M12_dv_change_df = np.log(dv_df.rolling(period_12m).sum()/dv_df.shift(period_12m).rolling(period_12m).sum())
M12_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.157187,0.061833,-0.050852,0.008753,0.061574,0.072704,0.129224,0.172907,0.394196,0.120074,...,-0.095902,-0.228237,0.291660,0.046147,0.130671,0.149549,0.215381,-0.061877,0.243743,0.074870
2024-06-07,0.151109,0.064797,-0.050808,0.001381,0.059318,0.076760,0.125165,0.172142,0.389869,0.116562,...,-0.092999,-0.241940,0.286624,0.048100,0.124286,0.141456,0.211436,-0.062041,0.242482,0.075323
2024-06-10,0.149116,0.063583,-0.047377,-0.000157,0.052623,0.077454,0.121620,0.171022,0.386267,0.114703,...,-0.092742,-0.251074,0.278176,0.051308,0.121759,0.139418,0.209808,-0.062684,0.238851,0.075403


In [149]:
#Calculate the percentage of positive returns over the past 1 month
M1_positive_returns_df = (daily_return_df\
.rolling(window=period_1m, min_periods=period_1m)\
.agg(lambda x: (x > 0).sum()))/period_1m

M1_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.476190,0.428571,0.809524,0.523810,0.380952,0.476190,0.476190,0.380952,0.428571,0.571429,...,0.428571,0.428571,0.523810,0.428571,0.571429,0.523810,0.428571,0.428571,0.571429,0.619048
2024-06-07,0.476190,0.428571,0.809524,0.571429,0.380952,0.523810,0.476190,0.333333,0.476190,0.523810,...,0.428571,0.476190,0.523810,0.428571,0.571429,0.476190,0.428571,0.380952,0.619048,0.619048
2024-06-10,0.476190,0.380952,0.761905,0.619048,0.428571,0.523810,0.476190,0.380952,0.476190,0.571429,...,0.428571,0.428571,0.523810,0.428571,0.571429,0.428571,0.380952,0.428571,0.619048,0.619048


In [150]:
#Calculate the percentage of positive returns over past 2 months
M2_positive_returns_df = (daily_return_df\
.rolling(window=period_2m, min_periods=period_2m)\
.agg(lambda x: (x > 0).sum()))/period_2m

M2_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.50000,0.50000,0.666667,0.50000,0.50000,0.428571,0.47619,0.452381,0.452381,0.595238,...,0.404762,0.428571,0.571429,0.452381,0.619048,0.547619,0.380952,0.476190,0.523810,0.571429
2024-06-07,0.50000,0.50000,0.666667,0.50000,0.47619,0.428571,0.50000,0.428571,0.452381,0.571429,...,0.380952,0.428571,0.547619,0.428571,0.619048,0.523810,0.357143,0.452381,0.523810,0.547619
2024-06-10,0.52381,0.50000,0.666667,0.52381,0.50000,0.428571,0.50000,0.452381,0.452381,0.595238,...,0.404762,0.428571,0.571429,0.428571,0.642857,0.500000,0.357143,0.476190,0.547619,0.571429


In [151]:
#Calculate the percentage of positive returns over past 3 months
M3_positive_returns_df = (daily_return_df\
.rolling(window=period_3m, min_periods=period_3m)\
.agg(lambda x: (x > 0).sum()))/period_3m

M3_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.460317,0.492063,0.619048,0.523810,0.460317,0.412698,0.507937,0.412698,0.412698,0.555556,...,0.412698,0.428571,0.587302,0.539683,0.571429,0.507937,0.396825,0.476190,0.47619,0.523810
2024-06-07,0.476190,0.507937,0.619048,0.539683,0.444444,0.412698,0.523810,0.412698,0.428571,0.555556,...,0.396825,0.444444,0.571429,0.523810,0.571429,0.507937,0.396825,0.476190,0.47619,0.523810
2024-06-10,0.492063,0.507937,0.603175,0.539683,0.460317,0.412698,0.523810,0.428571,0.412698,0.555556,...,0.396825,0.428571,0.571429,0.523810,0.587302,0.492063,0.380952,0.492063,0.47619,0.539683


In [152]:
#Calculate the percentage of positive returns over past 6 months
M6_positive_returns_df = (daily_return_df\
.rolling(window=period_6m, min_periods=period_6m)\
.agg(lambda x: (x > 0).sum()))/period_6m

M6_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.515625,0.484375,0.492188,0.601562,0.500000,0.515625,0.523438,0.484375,0.429688,0.570312,...,0.500000,0.515625,0.531250,0.515625,0.585938,0.554688,0.460938,0.492188,0.523438,0.570312
2024-06-07,0.515625,0.484375,0.500000,0.601562,0.500000,0.515625,0.531250,0.484375,0.437500,0.562500,...,0.500000,0.523438,0.523438,0.515625,0.585938,0.554688,0.460938,0.492188,0.523438,0.570312
2024-06-10,0.523438,0.484375,0.492188,0.601562,0.500000,0.515625,0.539062,0.492188,0.437500,0.570312,...,0.507812,0.523438,0.531250,0.523438,0.593750,0.554688,0.460938,0.500000,0.531250,0.578125


In [153]:
#Calculate the percentage of positive returns over past 12 months
M12_positive_returns_df = (daily_return_df\
.rolling(window=period_12m, min_periods=period_12m)\
.agg(lambda x: (x > 0).sum()))/period_12m

M12_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06,0.527344,0.480469,0.531250,0.562500,0.496094,0.503906,0.542969,0.523438,0.496094,0.535156,...,0.511719,0.519531,0.500000,0.523438,0.562500,0.539062,0.464844,0.519531,0.503906,0.562500
2024-06-07,0.527344,0.484375,0.531250,0.566406,0.492188,0.503906,0.546875,0.523438,0.496094,0.531250,...,0.507812,0.523438,0.500000,0.519531,0.562500,0.535156,0.460938,0.515625,0.503906,0.558594
2024-06-10,0.527344,0.480469,0.527344,0.566406,0.492188,0.500000,0.546875,0.523438,0.492188,0.531250,...,0.507812,0.519531,0.503906,0.519531,0.562500,0.531250,0.457031,0.515625,0.503906,0.558594


In [154]:
#Convert all the dataframes into a single column

Close=df1.stack(future_stack=True).rename("Close")
Daily_Returns=daily_return_df.stack(future_stack=True).rename("Daily_Returns")
M1_return=M1_return_df.stack(future_stack=True).rename("M1_return")
M2_return=M2_return_df.stack(future_stack=True).rename("M2_return")
M3_return=M3_return_df.stack(future_stack=True).rename("M3_return")
M6_return=M6_return_df.stack(future_stack=True).rename("M6_return")
M12_return=M12_return_df.stack(future_stack=True).rename("M12_return")

M1_forward_return=M1_forward_return_df.stack(future_stack=True).rename("M1_forward_return")

M1_sd=M1_sd_df.stack(future_stack=True).rename("M1_sd")
M2_sd=M2_sd_df.stack(future_stack=True).rename("M2_sd")
M3_sd=M3_sd_df.stack(future_stack=True).rename("M3_sd")
M6_sd=M6_sd_df.stack(future_stack=True).rename("M6_sd")
M12_sd=M12_sd_df.stack(future_stack=True).rename("M12_sd")

M12_1_return=M12_1_return_df.stack(future_stack=True).rename("M12_1_return")
M12_2_return=M12_2_return_df.stack(future_stack=True).rename("M12_2_return")
M12_3_return=M12_3_return_df.stack(future_stack=True).rename("M12_3_return")
M12_6_return=M12_6_return_df.stack(future_stack=True).rename("M12_6_return")

dv_change=dv_change_df.stack(future_stack=True).rename("dv_change")
M1_dv_change=M1_dv_change_df.stack(future_stack=True).rename("M1_dv_change")
M2_dv_change=M2_dv_change_df.stack(future_stack=True).rename("M2_dv_change")
M3_dv_change=M3_dv_change_df.stack(future_stack=True).rename("M3_dv_change")
M6_dv_change=M6_dv_change_df.stack(future_stack=True).rename("M6_dv_change")
M12_dv_change=M12_dv_change_df.stack(future_stack=True).rename("M12_dv_change")

M1_positive_returns=M1_positive_returns_df.stack(future_stack=True).rename("M1_positive_returns")
M2_positive_returns=M2_positive_returns_df.stack(future_stack=True).rename("M2_positive_returns")
M3_positive_returns=M3_positive_returns_df.stack(future_stack=True).rename("M3_positive_returns")
M6_positive_returns=M6_positive_returns_df.stack(future_stack=True).rename("M6_positive_returns")
M12_positive_returns=M12_positive_returns_df.stack(future_stack=True).rename("M12_positive_returns")

In [155]:
#Combine all the dataframes into one

Combined_df = pd.merge(Daily_Returns, M1_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M2_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M3_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M6_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, M1_forward_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, M1_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M2_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M3_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M6_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, M12_1_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_2_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_3_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_6_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M1_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M2_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M3_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M6_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, M1_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M2_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M3_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M6_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])


In [156]:
8677*504

4373208

In [157]:
Combined_df

Daily_Returns  M1_return  M2_return  M3_return  M6_return  \
Date       Ticker                                                              
1990-01-02 A                 NaN        NaN        NaN        NaN        NaN   
           AAL               NaN        NaN        NaN        NaN        NaN   
           AAPL              NaN        NaN        NaN        NaN        NaN   
           ABBV              NaN        NaN        NaN        NaN        NaN   
           ABNB              NaN        NaN        NaN        NaN        NaN   
...                          ...        ...        ...        ...        ...   
2024-06-12 YUM          0.002114   0.011625   0.001267  -0.002788   0.109606   
           ZBH         -0.016430  -0.107044  -0.147119  -0.162049  -0.083753   
           ZBRA         0.010496  -0.036874   0.050091   0.068135   0.266843   
           ZTS         -0.015181   0.046203   0.161977   0.000689  -0.028328   
           ^GSPC        0.008468   0.037516   0.056466   0.048321   0.167367   

                   M12_return  M1_forward_return     M1_sd     M2_sd  \
Date       Ticker                                                      
1990-01-02 A              NaN                NaN       NaN       NaN   
           AAL            NaN                NaN       NaN       NaN   
           AAPL           NaN           0.091292       NaN       NaN   
           ABBV           NaN                NaN       NaN       NaN   
           ABNB           NaN                NaN       NaN       NaN   
...                       ...                ...       ...       ...   
2024-06-12 YUM       0.052009                NaN  0.011440  0.011494   
           ZBH      -0.184882                NaN  0.011348  0.011304   
           ZBRA      0.120076                NaN  0.012930  0.016889   
           ZTS       0.042711                NaN  0.010190  0.015958   
           ^GSPC     0.235434                NaN  0.005474  0.006709   

                      M3_sd  ...  M1_dv_change  M2_dv_change  M3_dv_change  \
Date       Ticker            ...                                             
1990-01-02 A            NaN  ...           NaN           NaN           NaN   
           AAL          NaN  ...           NaN           NaN           NaN   
           AAPL         NaN  ...           NaN           NaN           NaN   
           ABBV         NaN  ...           NaN           NaN           NaN   
           ABNB         NaN  ...           NaN           NaN           NaN   
...                     ...  ...           ...           ...           ...   
2024-06-12 YUM     0.010769  ...     -0.286991     -0.033730      0.126890   
           ZBH     0.012140  ...      0.017594      0.024992     -0.252301   
           ZBRA    0.017044  ...     -0.384781     -0.083538     -0.085332   
           ZTS     0.018196  ...     -0.545957     -0.169119      0.215770   
           ^GSPC   0.006823  ...      0.033396     -0.040701      0.015796   

                   M6_dv_change  M12_dv_change  M1_positive_returns  \
Date       Ticker                                                     
1990-01-02 A                NaN            NaN                  NaN   
           AAL              NaN            NaN                  NaN   
           AAPL             NaN            NaN                  NaN   
           ABBV             NaN            NaN                  NaN   
           ABNB             NaN            NaN                  NaN   
...                         ...            ...                  ...   
2024-06-12 YUM         0.252515       0.138572             0.476190   
           ZBH        -0.055222       0.217608             0.380952   
           ZBRA       -0.086212      -0.065215             0.428571   
           ZTS         0.434612       0.240712             0.571429   
           ^GSPC       0.167001       0.077818             0.666667   

                   M2_positive_returns  M3_positive_returns  \
Date       Ticker                      

In [158]:
list(Combined_df.columns)

['Daily_Returns',
 'M1_return',
 'M2_return',
 'M3_return',
 'M6_return',
 'M12_return',
 'M1_forward_return',
 'M1_sd',
 'M2_sd',
 'M3_sd',
 'M6_sd',
 'M12_sd',
 'M12_1_return',
 'M12_2_return',
 'M12_3_return',
 'M12_6_return',
 'dv_change',
 'M1_dv_change',
 'M2_dv_change',
 'M3_dv_change',
 'M6_dv_change',
 'M12_dv_change',
 'M1_positive_returns',
 'M2_positive_returns',
 'M3_positive_returns',
 'M6_positive_returns',
 'M12_positive_returns']

In [159]:
#Obtain all the Market returns, forward returns and market standard deviation

Mkt_ret=Combined_df.xs("^GSPC", level="Ticker")['Daily_Returns'].rename("Mkt_ret")
Mkt_1m=Combined_df.xs("^GSPC", level="Ticker")['M1_return'].rename("Mkt_1m")
Mkt_2m=Combined_df.xs("^GSPC", level="Ticker")['M2_return'].rename("Mkt_2m")
Mkt_3m=Combined_df.xs("^GSPC", level="Ticker")['M3_return'].rename("Mkt_3m")
Mkt_6m=Combined_df.xs("^GSPC", level="Ticker")['M6_return'].rename("Mkt_6m")
Mkt_12m=Combined_df.xs("^GSPC", level="Ticker")['M12_return'].rename("Mkt_12m")

Mkt_M1_forward_return=Combined_df.xs("^GSPC", level="Ticker")['M1_forward_return'].rename("Mkt_M1_forward_return")

Mkt_std_1m=Combined_df.xs("^GSPC", level="Ticker")['M1_sd'].rename("Mkt_std_1m")
Mkt_std_2m=Combined_df.xs("^GSPC", level="Ticker")['M2_sd'].rename("Mkt_std_2m")
Mkt_std_3m=Combined_df.xs("^GSPC", level="Ticker")['M3_sd'].rename("Mkt_std_3m")
Mkt_std_6m=Combined_df.xs("^GSPC", level="Ticker")['M6_sd'].rename("Mkt_std_6m")
Mkt_std_12m=Combined_df.xs("^GSPC", level="Ticker")['M12_sd'].rename("Mkt_std_12m")

In [160]:
Mkt_M1_forward_return

Date
1990-01-02    0.088942
1990-01-03    0.087234
1990-01-04    0.072127
1990-01-05    0.059516
1990-01-08    0.070642
                ...   
2024-06-06         NaN
2024-06-07         NaN
2024-06-10         NaN
2024-06-11         NaN
2024-06-12         NaN
Name: Mkt_M1_forward_return, Length: 8678, dtype: float64

In [161]:
#Add the Market returns, forward returns and market standard deviation as columns into the new dataframe

Combined_df2 = pd.DataFrame(Mkt_ret).join(pd.DataFrame(Combined_df), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_1m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_2m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_3m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_6m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_12m).join(pd.DataFrame(Combined_df2), rsuffix='Date')

Combined_df2 = pd.DataFrame(Mkt_M1_forward_return).join(pd.DataFrame(Combined_df2), rsuffix='Date')

Combined_df2 = pd.DataFrame(Mkt_std_1m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_std_2m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_std_3m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_std_6m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_std_12m).join(pd.DataFrame(Combined_df2), rsuffix='Date')

In [162]:
Combined_df2

Mkt_std_12m  Mkt_std_6m  Mkt_std_3m  Mkt_std_2m  \
Date       Ticker                                                    
1990-01-02 A               NaN         NaN         NaN         NaN   
           AAL             NaN         NaN         NaN         NaN   
           AAPL            NaN         NaN         NaN         NaN   
           ABBV            NaN         NaN         NaN         NaN   
           ABNB            NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
2024-06-12 YUM        0.007113    0.006904    0.006823    0.006709   
           ZBH        0.007113    0.006904    0.006823    0.006709   
           ZBRA       0.007113    0.006904    0.006823    0.006709   
           ZTS        0.007113    0.006904    0.006823    0.006709   
           ^GSPC      0.007113    0.006904    0.006823    0.006709   

                   Mkt_std_1m  Mkt_M1_forward_return   Mkt_12m    Mkt_6m  \
Date       Ticker                                                          
1990-01-02 A              NaN               0.088942       NaN       NaN   
           AAL            NaN               0.088942       NaN       NaN   
           AAPL           NaN               0.088942       NaN       NaN   
           ABBV           NaN               0.088942       NaN       NaN   
           ABNB           NaN               0.088942       NaN       NaN   
...                       ...                    ...       ...       ...   
2024-06-12 YUM       0.005474                    NaN  0.235434  0.167367   
           ZBH       0.005474                    NaN  0.235434  0.167367   
           ZBRA      0.005474                    NaN  0.235434  0.167367   
           ZTS       0.005474                    NaN  0.235434  0.167367   
           ^GSPC     0.005474                    NaN  0.235434  0.167367   

                     Mkt_3m    Mkt_2m  ...  M1_dv_change  M2_dv_change  \
Date       Ticker                      ...                               
1990-01-02 A            NaN       NaN  ...           NaN           NaN   
           AAL          NaN       NaN  ...           NaN           NaN   
           AAPL         NaN       NaN  ...           NaN           NaN   
           ABBV         NaN       NaN  ...           NaN           NaN   
           ABNB         NaN       NaN  ...           NaN           NaN   
...                     ...       ...  ...           ...           ...   
2024-06-12 YUM     0.048321  0.056466  ...     -0.286991     -0.033730   
           ZBH     0.048321  0.056466  ...      0.017594      0.024992   
           ZBRA    0.048321  0.056466  ...     -0.384781     -0.083538   
           ZTS     0.048321  0.056466  ...     -0.545957     -0.169119   
           ^GSPC   0.048321  0.056466  ...      0.033396     -0.040701   

                   M3_dv_change  M6_dv_change  M12_dv_change  \
Date       Ticker                                              
1990-01-02 A                NaN           NaN            NaN   
           AAL              NaN           NaN            NaN   
           AAPL             NaN           NaN            NaN   
           ABBV             NaN           NaN            NaN   
           ABNB             NaN           NaN            NaN   
...                         ...           ...            ...   
2024-06-12 YUM         0.126890      0.252515       0.138572   
           ZBH        -0.252301     -0.055222       0.217608   
           ZBRA       -0.085332     -0.086212      -0.065215   
           ZTS         0.215770      0.434612       0.240712   
           ^GSPC       0.015796      0.167001       0.077818   

                   M1_positive_returns  M2_positive_returns  \
Date       Ticker                                             
1990-01-02 A                       NaN                  NaN   
           AAL                     NaN                  NaN   
           AAPL                    NaN                  NaN   
        

In [163]:
list(Combined_df2.columns)

['Mkt_std_12m',
 'Mkt_std_6m',
 'Mkt_std_3m',
 'Mkt_std_2m',
 'Mkt_std_1m',
 'Mkt_M1_forward_return',
 'Mkt_12m',
 'Mkt_6m',
 'Mkt_3m',
 'Mkt_2m',
 'Mkt_1m',
 'Mkt_ret',
 'Daily_Returns',
 'M1_return',
 'M2_return',
 'M3_return',
 'M6_return',
 'M12_return',
 'M1_forward_return',
 'M1_sd',
 'M2_sd',
 'M3_sd',
 'M6_sd',
 'M12_sd',
 'M12_1_return',
 'M12_2_return',
 'M12_3_return',
 'M12_6_return',
 'dv_change',
 'M1_dv_change',
 'M2_dv_change',
 'M3_dv_change',
 'M6_dv_change',
 'M12_dv_change',
 'M1_positive_returns',
 'M2_positive_returns',
 'M3_positive_returns',
 'M6_positive_returns',
 'M12_positive_returns']

In [164]:
#Drop ^GSPC (which is S&P 500) from the rows of the dataframe

Final_df=Combined_df2.drop('^GSPC', level='Ticker')

In [165]:
Final_df

Mkt_std_12m  Mkt_std_6m  Mkt_std_3m  Mkt_std_2m  \
Date       Ticker                                                    
1990-01-02 A               NaN         NaN         NaN         NaN   
           AAL             NaN         NaN         NaN         NaN   
           AAPL            NaN         NaN         NaN         NaN   
           ABBV            NaN         NaN         NaN         NaN   
           ABNB            NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
2024-06-12 XYL        0.007113    0.006904    0.006823    0.006709   
           YUM        0.007113    0.006904    0.006823    0.006709   
           ZBH        0.007113    0.006904    0.006823    0.006709   
           ZBRA       0.007113    0.006904    0.006823    0.006709   
           ZTS        0.007113    0.006904    0.006823    0.006709   

                   Mkt_std_1m  Mkt_M1_forward_return   Mkt_12m    Mkt_6m  \
Date       Ticker                                                          
1990-01-02 A              NaN               0.088942       NaN       NaN   
           AAL            NaN               0.088942       NaN       NaN   
           AAPL           NaN               0.088942       NaN       NaN   
           ABBV           NaN               0.088942       NaN       NaN   
           ABNB           NaN               0.088942       NaN       NaN   
...                       ...                    ...       ...       ...   
2024-06-12 XYL       0.005474                    NaN  0.235434  0.167367   
           YUM       0.005474                    NaN  0.235434  0.167367   
           ZBH       0.005474                    NaN  0.235434  0.167367   
           ZBRA      0.005474                    NaN  0.235434  0.167367   
           ZTS       0.005474                    NaN  0.235434  0.167367   

                     Mkt_3m    Mkt_2m  ...  M1_dv_change  M2_dv_change  \
Date       Ticker                      ...                               
1990-01-02 A            NaN       NaN  ...           NaN           NaN   
           AAL          NaN       NaN  ...           NaN           NaN   
           AAPL         NaN       NaN  ...           NaN           NaN   
           ABBV         NaN       NaN  ...           NaN           NaN   
           ABNB         NaN       NaN  ...           NaN           NaN   
...                     ...       ...  ...           ...           ...   
2024-06-12 XYL     0.048321  0.056466  ...      0.183076      0.394658   
           YUM     0.048321  0.056466  ...     -0.286991     -0.033730   
           ZBH     0.048321  0.056466  ...      0.017594      0.024992   
           ZBRA    0.048321  0.056466  ...     -0.384781     -0.083538   
           ZTS     0.048321  0.056466  ...     -0.545957     -0.169119   

                   M3_dv_change  M6_dv_change  M12_dv_change  \
Date       Ticker                                              
1990-01-02 A                NaN           NaN            NaN   
           AAL              NaN           NaN            NaN   
           AAPL             NaN           NaN            NaN   
           ABBV             NaN           NaN            NaN   
           ABNB             NaN           NaN            NaN   
...                         ...           ...            ...   
2024-06-12 XYL         0.133014      0.055950       0.109104   
           YUM         0.126890      0.252515       0.138572   
           ZBH        -0.252301     -0.055222       0.217608   
           ZBRA       -0.085332     -0.086212      -0.065215   
           ZTS         0.215770      0.434612       0.240712   

                   M1_positive_returns  M2_positive_returns  \
Date       Ticker                                             
1990-01-02 A                       NaN                  NaN   
           AAL                     NaN                  NaN   
           AAPL                    NaN                  NaN   
        

In [166]:
#Download data as csv if needed

#Final_df.to_csv(path_or_buf='/Users/bennyang/Downloads/S&P500 features.csv', index=True) 

In [167]:
Ticker_dict={"NVDA":"Nvidia",
             "META":"Meta",
             "TSLA":"Tesla",
             "AMZN":"Amazon",
             "GOOG":"Google",
             "MSFT":"Microsoft",
             "AAPL":"Apple",
             "GS":"Goldman",
             "JPM":"JPMorgan",
             "NFLX":"Netflix",
             "NKE":"Nike",
             "AAL":"AmericanAirlines",
             "LUV":"SouthwestAirlines",
             "GM":"GeneralMotors"}

In [168]:
#Download multiple tickers to csv
Ticker_list=["NVDA","META","TSLA","AMZN","GOOG","MSFT","AAPL","GS","JPM","NFLX","NKE","AAL","LUV","GM"]

In [169]:
#create the individual dataframes inside a dictionary
dfs = {}

for ticker in Ticker_list:
    dfs[ticker] = Final_df.xs(ticker, level="Ticker")
    

In [171]:
#export the dataframes inside a dictionary as csv
for ticker, data in dfs.items():
    data.to_csv(path_or_buf=f"/Users/bennyang/Downloads/{ticker} features.csv",index=True)

In [69]:
#Download single ticker to csv

#Apple=Final_df.xs("AAPL", level="Ticker")

In [84]:
#Apple

,Mkt_std_12m,Mkt_std_6m,Mkt_std_3m,Mkt_std_2m,Mkt_std_1m,Mkt_M1_forward_return,Mkt_12m,Mkt_6m,Mkt_3m,Mkt_2m,...,M1_dv_change,M2_dv_change,M3_dv_change,M6_dv_change,M12_dv_change,M1_positive_returns,M2_positive_returns,M3_positive_returns,M6_positive_returns,M12_positive_returns
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,0.088942,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,0.087234,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,0.072127,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,0.059516,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,0.070642,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-05,0.007183,0.006948,0.007045,0.007288,0.005283,NaN,0.241452,0.158818,0.047676,0.028357,...,-0.142346,-0.019268,-0.000054,0.038850,-0.046992,0.857143,0.666667,0.619048,0.500000,0.531250
2024-06-06,0.007169,0.006937,0.006940,0.007288,0.005297,NaN,0.247379,0.152761,0.037225,0.028532,...,-0.203826,-0.011258,-0.005009,0.045038,-0.050852,0.809524,0.666667,0.619048,0.492188,0.531250
2024-06-07,0.007149,0.006915,0.006883,0.007292,0.005318,NaN,0.236457,0.157069,0.042659,0.025971,...,-0.196990,0.002357,-0.016208,0.049828,-0.050808,0.809524,0.666667,0.619048,0.500000,0.531250


In [85]:
#Apple.to_csv(path_or_buf='/Users/bennyang/Downloads/Apple features.csv', index=True) 